# Vector Context Provider

In this notebook, you'll use **vector search** (semantic similarity) instead of fulltext (keyword) search for context retrieval. Vector search understands meaning — it can find conceptually related content even when exact keywords don't match.

***

Load the environment variables and import the required modules.

In [ ]:
import sys
sys.path.insert(0, '../shared')

from azure.identity import DefaultAzureCredential
from azure.identity.aio import AzureCliCredential

from agent_framework.azure import AzureAIClient
from agent_framework_neo4j import (
    AzureAIEmbedder,
    AzureAISettings,
    Neo4jContextProvider,
    Neo4jSettings,
)

from config import get_agent_config

Get the configuration and create the embedder for vector search.

In [ ]:
# Get configuration from environment
config = get_agent_config()
neo4j_settings = Neo4jSettings()
azure_settings = AzureAISettings()

# Create embedder for converting queries to vectors
sync_credential = DefaultAzureCredential()

embedder = AzureAIEmbedder(
    endpoint=azure_settings.inference_endpoint,
    credential=sync_credential,
    model=azure_settings.embedding_model,
)

print(f"Neo4j URI: {neo4j_settings.uri}")
print(f"Vector Index: {neo4j_settings.vector_index_name}")
print(f"Embedding Model: {azure_settings.embedding_model}")

## Create the Vector Context Provider

Create a `Neo4jContextProvider` with `index_type="vector"`. The provider will:
1. Embed the user's query using the Azure AI embedder
2. Search the Neo4j vector index for semantically similar chunks
3. Inject the results into the agent's context

In [ ]:
# Create context provider with vector search
provider = Neo4jContextProvider(
    uri=neo4j_settings.uri,
    username=neo4j_settings.username,
    password=neo4j_settings.get_password(),
    index_name=neo4j_settings.vector_index_name,
    index_type="vector",
    embedder=embedder,
    top_k=5,
    context_prompt=(
        "## Semantic Search Results\n"
        "Use the following semantically relevant information from the "
        "knowledge graph to answer questions:"
    ),
)

> Vector search requires an `embedder` to convert queries into vectors. The `index_type="vector"` tells the provider to use the `VectorRetriever` from neo4j-graphrag.

***

## Run the Agent with Vector Context

Create an agent that receives automatically injected semantic search results. The agent is created once and reused for all queries below.

In [ ]:
# Connect provider and create agent
credential = AzureCliCredential()
await provider.__aenter__()
print("Connected to Neo4j with vector search!\n")

client = AzureAIClient(
    project_endpoint=config.project_endpoint,
    model_deployment_name=config.model_name,
    credential=credential,
)

agent = client.as_agent(
    name="workshop-vector-agent",
    instructions=(
        "You are a helpful assistant that answers questions about companies "
        "using the provided semantic search context. Be concise and accurate. "
        "When the context contains relevant information, cite it in your response."
    ),
    context_providers=[provider],
)

session = agent.create_session()

async def ask(query):
    """Run a query against the agent and print the response."""
    print(f"User: {query}\n")
    print("Assistant: ", end="", flush=True)
    response = await agent.run(query, session=session)
    print(response.text)
    print()

print(f"Agent '{agent.name}' created")

In [ ]:
await ask("What are the main business activities of tech companies?")

> Vector search found semantically similar content even without exact keyword matches. Try queries that describe concepts rather than specific terms.

***

Run the cells below to experiment with different queries. You can also modify the queries or add new cells.

***

[View the complete code](../financial_data_load/solution_srcs/06_02_vector_context_provider.py)

[Move on to the Graph-Enriched Provider Notebook](02_graph_enriched_provider.ipynb)

In [ ]:
await ask("Describe challenges and risks in the technology sector")

In [ ]:
await ask("How do companies generate revenue and measure performance?")

In [ ]:
# Cleanup
await provider.__aexit__(None, None, None)
await credential.close()
embedder.close()